## Paquetes y datos originales ## 

In [109]:
import json
import pandas as pd
from datetime import datetime
from listas import disfagia_list, PA_list, P_list
from auxiliary_functions import valores_codigos, contar_diccionarios, dias_ingreso_total, asignar_intervalo_edad, sumar_barthel, sumar_emina, obtener_ultimo_resultat, obtener_valor_promedio, canadenca_comparada, disfagia_mecvvs, extraer_valor_clave, extraer_valor_clave_simple

tabla = pd.read_pickle('./data/processed/dataframe.pkl')

### Previamente se ha tratado la base de datos y se han hecho 16 nuevas columnas para facilitar la lectura

In [110]:
tabla.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,EMINA_sumatorios_comparados,emina_resultats,mna_resultats,promedio_pes,Canadenca_sumatorios_comparados,Disfagia_mecvvs,alteracioEficacia_mecvvs,alteracioSeguretat_mecvvs,viscositat_mecvvs,volumn_mecvvs
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,10.0,10.0,None,NaN,NaN,NaN,NaN,NaN,None,None
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,1.0,1.0,None,58.308333,NaN,0.0,1.0,0.0,Líquid,Alt
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,6.0,6.0,None,NaN,NaN,NaN,NaN,NaN,None,None
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,8.0,8.0,None,NaN,NaN,1.0,1.0,1.0,NECTAR,ALT
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,7.0,7.0,4.0,49.900000,NaN,1.0,NaN,NaN,None,None


## Dividiremos los datos en 3 grupos:
- Con PA (codigo ICD de PA )
- Con PA + MECVV (MECVV positivo + icd pneumonia < 30 days)
- Sin PA (ICD pneumonia or icd pneumonia + mecvv positive > 30 days)

# CON PA 

In [111]:
tabla.PA_diagnosticada.value_counts()

PA_diagnosticada
0.0    33
1.0    17
Name: count, dtype: int64

In [112]:
CON_PA = tabla[tabla['PA_diagnosticada'] == '1.0']
CON_PA.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,EMINA_sumatorios_comparados,emina_resultats,mna_resultats,promedio_pes,Canadenca_sumatorios_comparados,Disfagia_mecvvs,alteracioEficacia_mecvvs,alteracioSeguretat_mecvvs,viscositat_mecvvs,volumn_mecvvs


# CON_PA + MECVV

In [113]:
CON_PA_MECVV = tabla[(tabla['Disfagia_mecvvs'] == '1.0') & (tabla['P_diagnosticada'] == '1.0') & (tabla['Dias_totales_ingresado'] < 30)]
CON_PA.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,EMINA_sumatorios_comparados,emina_resultats,mna_resultats,promedio_pes,Canadenca_sumatorios_comparados,Disfagia_mecvvs,alteracioEficacia_mecvvs,alteracioSeguretat_mecvvs,viscositat_mecvvs,volumn_mecvvs


# SIN_PA

In [114]:
SIN_PA = tabla[(tabla['P_diagnosticada'] == '1.0') | (tabla['P_diagnosticada'] == '1.0') & (tabla['Disfagia_mecvvs'] == '1.0') & (tabla['Dias_totales_ingresado'] > 30)]
SIN_PA.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,EMINA_sumatorios_comparados,emina_resultats,mna_resultats,promedio_pes,Canadenca_sumatorios_comparados,Disfagia_mecvvs,alteracioEficacia_mecvvs,alteracioSeguretat_mecvvs,viscositat_mecvvs,volumn_mecvvs
